In [1]:
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium_stealth import stealth

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import json
import re
import random
from datetime import datetime, timedelta
from fake_useragent import UserAgent
import logging
from typing import List, Dict, Optional
import os

# Configuration du logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configuration globale
ua = UserAgent()

# Liste de proxies gratuits (remplacer par des proxies premium en production)
PROXY_LIST = [
    # "http://proxy1:port",
    # "http://proxy2:port", 
    # Ajoutez vos proxies ici
]

# User agents réalistes
REALISTIC_USER_AGENTS = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/121.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.1 Safari/605.1.15'
]

print("✅ Imports terminés - Système anti-détection prêt ! 🕵️")

✅ Imports terminés - Système anti-détection prêt ! 🕵️


In [2]:
# ============================================================================
# SCOUT ROBUSTE POUR DÉTECTION DE BALISES
# ============================================================================

class RobustProductReviewScout:
    """
    Scout ultra-robuste pour détecter automatiquement les balises de produits et reviews
    Avec gestion d'erreurs complète et fallbacks multiples
    """
    
    def __init__(self):
        self.driver = None
        self.detected_selectors = {}
        self.base_selectors = {
            'amazon': {
                'products': {
                    'containers': [
                        '[data-component-type="s-search-result"]',
                        '.s-result-item',
                        '.s-widget-container .s-card-container',
                        '[data-asin]:not([data-asin=""])'
                    ],
                    'titles': [
                        'h2 span',
                        'h2 a span',
                        '.s-size-mini span',
                        '.a-size-base-plus',
                        '[data-cy="title-recipe-title"]'
                    ],
                    'urls': [
                        'h2 a',
                        '.a-link-normal',
                        'a[href*="/dp/"]',
                        'a[href*="/gp/product/"]'
                    ],
                    'prices': [
                        '.a-price .a-offscreen',
                        '.a-price-whole',
                        '.a-price-range .a-offscreen',
                        '.a-price-symbol + .a-price-whole'
                    ],
                    'ratings': [
                        '.a-icon-alt',
                        '.a-star-mini .a-icon-alt',
                        'span[aria-label*="stars"]'
                    ]
                },
                'reviews': {
                    'containers': [
                        '[data-hook="review"]',
                        '.review',
                        '.cr-original-review-content',
                        '.reviewText'
                    ],
                    'titles': [
                        '[data-hook="review-title"] span',
                        '.review-title',
                        '.cr-original-review-content .review-title'
                    ],
                    'texts': [
                        '[data-hook="review-body"] span',
                        '.review-text',
                        '.cr-original-review-content .review-text',
                        '.reviewText'
                    ],
                    'ratings': [
                        '[data-hook="review-star-rating"] .a-icon-alt',
                        '.review-rating .a-icon-alt',
                        '.cr-original-review-content .a-icon-alt'
                    ],
                    'authors': [
                        '.a-profile-name',
                        '.review-author',
                        '.cr-original-review-content .author'
                    ],
                    'dates': [
                        '[data-hook="review-date"]',
                        '.review-date',
                        '.cr-original-review-content .review-date'
                    ]
                }
            },
            'ebay': {
                'products': {
                    'containers': [
                        '.s-item',
                        '.srp-results .s-item',
                        '.b-listing__wrap'
                    ],
                    'titles': [
                        '.s-item__title',
                        '.it-ttl',
                        '.b-listing__title'
                    ],
                    'urls': [
                        '.s-item__link',
                        '.it-ttl a',
                        '.b-listing__title a'
                    ],
                    'prices': [
                        '.s-item__price',
                        '.notranslate',
                        '.b-listing__price'
                    ]
                },
                'reviews': {
                    'containers': [
                        '.review-item',
                        '.ebay-review',
                        '.reviews .review'
                    ],
                    'texts': [
                        '.review-item-content',
                        '.ebay-review-text',
                        '.review-content'
                    ]
                }
            }
        }
    
    def setup_robust_driver(self, headless=True, timeout=30):
        """Configuration driver ultra-robuste avec fallbacks"""
        
        max_attempts = 3
        
        for attempt in range(max_attempts):
            try:
                print(f"🔧 Tentative {attempt + 1}/{max_attempts} - Setup driver scout...")
                
                # Fermer le driver existant si nécessaire
                if self.driver:
                    try:
                        self.driver.quit()
                    except:
                        pass
                    self.driver = None
                
                # Options ultra-sûres
                options = self._create_safe_options(headless)
                
                # Création driver avec timeout
                import undetected_chromedriver as uc
                
                self.driver = uc.Chrome(
                    options=options,
                    version_main=None,
                    headless=headless,
                    use_subprocess=False,
                    log_level=3
                )
                
                # Configuration des timeouts
                self.driver.set_page_load_timeout(timeout)
                self.driver.implicitly_wait(10)
                
                # Test de fonctionnement
                self.driver.get("data:text/html,<html><body><h1>Test Scout</h1></body></html>")
                
                print("✅ Driver scout initialisé avec succès!")
                return True
                
            except Exception as e:
                print(f"❌ Tentative {attempt + 1} échouée: {str(e)[:100]}...")
                if attempt < max_attempts - 1:
                    time.sleep(3)
                else:
                    print("❌ Impossible de créer le driver scout")
                    return False
        
        return False
    
    def _create_safe_options(self, headless=True):
        """Crée des options Chrome ultra-sûres"""
        
        import undetected_chromedriver as uc
        
        options = uc.ChromeOptions()
        
        # Arguments de base seulement
        safe_args = [
            '--no-sandbox',
            '--disable-dev-shm-usage',
            '--disable-gpu',
            '--disable-web-security',
            '--disable-features=VizDisplayCompositor',
            '--window-size=1920,1080',
            '--start-maximized'
        ]
        
        for arg in safe_args:
            options.add_argument(arg)
        
        if headless:
            options.add_argument('--headless=new')
        
        # User agent aléatoire
        try:
            user_agent = random.choice(REALISTIC_USER_AGENTS)
            options.add_argument(f'--user-agent={user_agent}')
        except:
            options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')
        
        # Préférences minimales
        prefs = {
            "profile.default_content_setting_values.notifications": 2,
            "profile.default_content_settings.popups": 0
        }
        options.add_experimental_option("prefs", prefs)
        
        return options
    
    def detect_site_selectors(self, site_url, search_term="laptop", max_retries=3):
        """
        Détecte automatiquement tous les sélecteurs pour un site
        """
        
        if not self.driver:
            print("❌ Driver non initialisé")
            return {}
        
        site_type = 'amazon' if 'amazon' in site_url else 'ebay' if 'ebay' in site_url else 'unknown'
        
        if site_type == 'unknown':
            print(f"❌ Site non supporté: {site_url}")
            return {}
        
        print(f"🔍 Détection des sélecteurs pour {site_type}...")
        
        detected = {
            'site': site_type,
            'products': {},
            'reviews': {}
        }
        
        try:
            # Phase 1: Détection sélecteurs produits
            product_selectors = self._detect_product_selectors(site_url, search_term, site_type)
            detected['products'] = product_selectors
            
            if product_selectors:
                print(f"✅ Sélecteurs produits détectés: {len(product_selectors)}")
                
                # Phase 2: Détection sélecteurs reviews
                review_selectors = self._detect_review_selectors(site_type, product_selectors)
                detected['reviews'] = review_selectors
                
                if review_selectors:
                    print(f"✅ Sélecteurs reviews détectés: {len(review_selectors)}")
                else:
                    print("⚠️ Sélecteurs reviews non détectés, utilisation des défauts")
                    detected['reviews'] = self.base_selectors[site_type]['reviews']
            else:
                print("❌ Impossible de détecter les sélecteurs produits")
                return {}
            
            # Sauvegarde des sélecteurs
            self._save_detected_selectors(detected, site_type)
            
            return detected
            
        except Exception as e:
            print(f"❌ Erreur détection: {e}")
            return {}
    
    def _detect_product_selectors(self, site_url, search_term, site_type):
        """Détecte les sélecteurs de produits avec tests multiples"""
        
        try:
            # Construire URL de recherche
            if site_type == 'amazon':
                search_url = f"https://www.amazon.com/s?k={search_term.replace(' ', '+')}"
            elif site_type == 'ebay':
                search_url = f"https://www.ebay.com/sch/i.html?_nkw={search_term.replace(' ', '+')}"
            else:
                return {}
            
            print(f"🌐 Navigation vers: {search_url}")
            self.driver.get(search_url)
            
            # Attendre le chargement
            time.sleep(5)
            
            # Tester les sélecteurs de conteneurs
            selectors = {}
            base_selectors = self.base_selectors[site_type]['products']
            
            # Test conteneurs de produits
            for selector in base_selectors['containers']:
                try:
                    elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                    if len(elements) >= 3:  # Au moins 3 produits
                        selectors['container'] = selector
                        print(f"✅ Conteneur: {selector} ({len(elements)} éléments)")
                        break
                except:
                    continue
            
            if not selectors.get('container'):
                print("❌ Aucun conteneur de produit trouvé")
                return {}
            
            # Test autres sélecteurs dans le contexte du conteneur
            container_elements = self.driver.find_elements(By.CSS_SELECTOR, selectors['container'])
            
            if container_elements:
                first_container = container_elements[0]
                
                # Test titres
                for title_selector in base_selectors['titles']:
                    try:
                        title_elem = first_container.find_element(By.CSS_SELECTOR, title_selector)
                        if title_elem.text.strip():
                            selectors['title'] = title_selector
                            print(f"✅ Titre: {title_selector}")
                            break
                    except:
                        continue
                
                # Test URLs
                for url_selector in base_selectors['urls']:
                    try:
                        url_elem = first_container.find_element(By.CSS_SELECTOR, url_selector)
                        href = url_elem.get_attribute('href')
                        if href and ('amazon.com' in href or 'ebay.com' in href):
                            selectors['url'] = url_selector
                            print(f"✅ URL: {url_selector}")
                            break
                    except:
                        continue
                
                # Test prix
                for price_selector in base_selectors['prices']:
                    try:
                        price_elem = first_container.find_element(By.CSS_SELECTOR, price_selector)
                        if price_elem.text.strip() and ('$' in price_elem.text or '€' in price_elem.text):
                            selectors['price'] = price_selector
                            print(f"✅ Prix: {price_selector}")
                            break
                    except:
                        continue
                
                # Test ratings
                for rating_selector in base_selectors['ratings']:
                    try:
                        rating_elem = first_container.find_element(By.CSS_SELECTOR, rating_selector)
                        rating_text = rating_elem.get_attribute('textContent') or rating_elem.text
                        if rating_text and ('star' in rating_text.lower() or 'étoile' in rating_text.lower()):
                            selectors['rating'] = rating_selector
                            print(f"✅ Rating: {rating_selector}")
                            break
                    except:
                        continue
            
            return selectors
            
        except Exception as e:
            print(f"❌ Erreur détection produits: {e}")
            return {}
    
    def _detect_review_selectors(self, site_type, product_selectors):
        """Détecte les sélecteurs de reviews en naviguant vers un produit"""
        
        try:
            # Trouver un lien produit
            if not product_selectors.get('url'):
                print("❌ Pas de sélecteur URL disponible")
                return {}
            
            # Récupérer le premier lien produit
            product_links = self.driver.find_elements(By.CSS_SELECTOR, product_selectors['url'])
            
            if not product_links:
                print("❌ Aucun lien produit trouvé")
                return {}
            
            product_url = product_links[0].get_attribute('href')
            print(f"🔗 Test reviews sur: {product_url[:80]}...")
            
            # Naviguer vers le produit
            self.driver.get(product_url)
            time.sleep(3)
            
            # Chercher les reviews sur la page produit ou naviguer vers la page reviews
            review_selectors = {}
            base_selectors = self.base_selectors[site_type]['reviews']
            
            # D'abord, chercher un lien vers les reviews
            review_link_selectors = [
                'a[href*="customer-reviews"]',
                'a[href*="reviews"]',
                'a[href*="review"]',
                '.cr-widget-ACR a'
            ]
            
            review_page_found = False
            for link_selector in review_link_selectors:
                try:
                    review_links = self.driver.find_elements(By.CSS_SELECTOR, link_selector)
                    for link in review_links:
                        href = link.get_attribute('href')
                        if href and 'review' in href:
                            print(f"🔗 Navigation vers page reviews: {href[:80]}...")
                            self.driver.get(href)
                            time.sleep(3)
                            review_page_found = True
                            break
                    if review_page_found:
                        break
                except:
                    continue
            
            # Tester les sélecteurs de reviews
            # Test conteneurs
            for container_selector in base_selectors['containers']:
                try:
                    review_elements = self.driver.find_elements(By.CSS_SELECTOR, container_selector)
                    if len(review_elements) >= 2:  # Au moins 2 reviews
                        review_selectors['container'] = container_selector
                        print(f"✅ Conteneur reviews: {container_selector} ({len(review_elements)} reviews)")
                        break
                except:
                    continue
            
            if review_selectors.get('container'):
                # Tester les autres sélecteurs dans le contexte
                review_containers = self.driver.find_elements(By.CSS_SELECTOR, review_selectors['container'])
                
                if review_containers:
                    first_review = review_containers[0]
                    
                    # Test texte de review
                    for text_selector in base_selectors['texts']:
                        try:
                            text_elem = first_review.find_element(By.CSS_SELECTOR, text_selector)
                            if text_elem.text.strip() and len(text_elem.text.strip()) > 20:
                                review_selectors['text'] = text_selector
                                print(f"✅ Texte review: {text_selector}")
                                break
                        except:
                            continue
                    
                    # Test titre de review
                    for title_selector in base_selectors['titles']:
                        try:
                            title_elem = first_review.find_element(By.CSS_SELECTOR, title_selector)
                            if title_elem.text.strip():
                                review_selectors['title'] = title_selector
                                print(f"✅ Titre review: {title_selector}")
                                break
                        except:
                            continue
                    
                    # Test rating
                    for rating_selector in base_selectors['ratings']:
                        try:
                            rating_elem = first_review.find_element(By.CSS_SELECTOR, rating_selector)
                            rating_text = rating_elem.get_attribute('textContent') or rating_elem.text
                            if rating_text and ('star' in rating_text.lower() or 'étoile' in rating_text.lower()):
                                review_selectors['rating'] = rating_selector
                                print(f"✅ Rating review: {rating_selector}")
                                break
                        except:
                            continue
                    
                    # Test auteur
                    for author_selector in base_selectors['authors']:
                        try:
                            author_elem = first_review.find_element(By.CSS_SELECTOR, author_selector)
                            if author_elem.text.strip():
                                review_selectors['author'] = author_selector
                                print(f"✅ Auteur review: {author_selector}")
                                break
                        except:
                            continue
                    
                    # Test date
                    for date_selector in base_selectors['dates']:
                        try:
                            date_elem = first_review.find_element(By.CSS_SELECTOR, date_selector)
                            if date_elem.text.strip():
                                review_selectors['date'] = date_selector
                                print(f"✅ Date review: {date_selector}")
                                break
                        except:
                            continue
            
            return review_selectors
            
        except Exception as e:
            print(f"❌ Erreur détection reviews: {e}")
            return {}
    
    def _save_detected_selectors(self, selectors, site_type):
        """Sauvegarde les sélecteurs détectés"""
        
        try:
            import os
            import json
            
            config_dir = "../config"
            os.makedirs(config_dir, exist_ok=True)
            
            filename = f"{config_dir}/detected_selectors_{site_type}.json"
            
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(selectors, f, indent=2, ensure_ascii=False)
            
            print(f"✅ Sélecteurs sauvegardés: {filename}")
            return True
            
        except Exception as e:
            print(f"❌ Erreur sauvegarde: {e}")
            return False
    
    def close(self):
        """Ferme proprement le driver"""
        if self.driver:
            try:
                self.driver.quit()
                print("✅ Driver scout fermé")
            except:
                pass
            self.driver = None

print("✅ Scout robuste créé!")

✅ Scout robuste créé!


In [6]:
# ============================================================================
# SCRAPER ROBUSTE POUR REVIEWS DE PRODUITS
# ============================================================================

class RobustProductReviewScraper:
    """
    Scraper ultra-robuste pour récupérer les reviews de produits
    Utilise les sélecteurs détectés par le scout
    """
    
    def __init__(self, selectors_file=None):
        self.driver = None
        self.selectors = {}
        self.scraped_data = []
        self.session_stats = {
            'products_processed': 0,
            'reviews_collected': 0,
            'errors': 0,
            'start_time': None
        }
        
        if selectors_file:
            self.load_selectors(selectors_file)
    
    def load_selectors(self, filename):
        """Charge les sélecteurs depuis un fichier JSON"""
        try:
            import json
            with open(filename, 'r', encoding='utf-8') as f:
                self.selectors = json.load(f)
            print(f"✅ Sélecteurs chargés: {filename}")
            return True
        except Exception as e:
            print(f"❌ Erreur chargement sélecteurs: {e}")
            return False
    
    def setup_robust_driver(self, headless=False, timeout=60):
        """Configuration driver ultra-robuste pour scraping"""
        
        max_attempts = 3
        
        for attempt in range(max_attempts):
            try:
                print(f"🚀 Tentative {attempt + 1}/{max_attempts} - Setup driver scraper...")
                
                # Fermer le driver existant
                if self.driver:
                    try:
                        self.driver.quit()
                    except:
                        pass
                    self.driver = None
                
                # Options anti-détection
                options = self._create_stealth_options(headless)
                
                # Création du driver
                import undetected_chromedriver as uc
                
                self.driver = uc.Chrome(
                    options=options,
                    version_main=None,
                    use_subprocess=False,
                    log_level=3
                )
                
                # Configuration timeouts
                self.driver.set_page_load_timeout(timeout)
                self.driver.implicitly_wait(15)
                
                # Scripts anti-détection
                self._inject_stealth_scripts()
                
                # Test fonctionnement
                self.driver.get("data:text/html,<html><body><h1>Scraper Ready</h1></body></html>")
                
                print("✅ Driver scraper prêt!")
                print(f"🎭 User-Agent: {self.driver.execute_script('return navigator.userAgent;')[:80]}...")
                
                return True
                
            except Exception as e:
                print(f"❌ Tentative {attempt + 1} échouée: {str(e)[:100]}...")
                if attempt < max_attempts - 1:
                    time.sleep(5)
                else:
                    print("❌ Impossible de créer le driver scraper")
                    return False
        
        return False
    
    def _create_stealth_options(self, headless=True):
        """Crée des options Chrome furtives"""
        
        import undetected_chromedriver as uc
        
        options = uc.ChromeOptions()
        
        # Arguments furtifs
        stealth_args = [
            '--no-sandbox',
            '--disable-dev-shm-usage',
            '--disable-gpu',
            '--disable-web-security',
            '--disable-features=VizDisplayCompositor',
            '--disable-blink-features=AutomationControlled',
            '--disable-extensions',
            '--no-first-run',
            '--no-default-browser-check',
            '--disable-default-apps',
            '--disable-background-timer-throttling',
            '--disable-backgrounding-occluded-windows',
            '--disable-renderer-backgrounding',
            '--window-size=1920,1080',
            '--start-maximized'
        ]
        
        for arg in stealth_args:
            options.add_argument(arg)
        
        if headless:
            options.add_argument('--headless=new')
        
        # User agent aléatoire réaliste
        try:
            user_agent = random.choice(REALISTIC_USER_AGENTS)
            options.add_argument(f'--user-agent={user_agent}')
        except:
            options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
        
        # Préférences furtives
        prefs = {
            "profile.default_content_setting_values.notifications": 2,
            "profile.default_content_settings.popups": 0,
            "profile.managed_default_content_settings.images": 1,  # Charger les images
            "profile.default_content_setting_values.plugins": 1,
            "profile.content_settings.plugin_whitelist.adobe-flash-player": 1,
            "profile.content_settings.exceptions.plugins.*,*.per_resource.adobe-flash-player": 1
        }
        options.add_experimental_option("prefs", prefs)
        
        return options
    
    def _inject_stealth_scripts(self):
        """Injecte des scripts anti-détection"""
        try:
            stealth_script = '''
                Object.defineProperty(navigator, 'webdriver', {
                    get: () => undefined,
                });
                
                Object.defineProperty(navigator, 'plugins', {
                    get: () => [1, 2, 3, 4, 5],
                });
                
                Object.defineProperty(navigator, 'languages', {
                    get: () => ['en-US', 'en'],
                });
                
                window.chrome = {
                    runtime: {},
                };
                
                Object.defineProperty(navigator, 'permissions', {
                    get: () => ({
                        query: () => Promise.resolve({ state: 'granted' }),
                    }),
                });
            '''
            
            self.driver.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {
                'source': stealth_script
            })
            
        except Exception as e:
            print(f"⚠️ Scripts anti-détection non injectés: {e}")
    
    def scrape_category_reviews(self, category, site='amazon', max_products=10, reviews_per_rating=50):
        """
        Scrape principal pour récupérer les reviews d'une catégorie
        """
        
        if not self.driver:
            print("❌ Driver non initialisé")
            return pd.DataFrame()
        
        if not self.selectors:
            print("❌ Sélecteurs non chargés")
            return pd.DataFrame()
        
        print("="*80)
        print("🎯 DÉBUT DU SCRAPING ROBUSTE")
        print("="*80)
        print(f"📦 Catégorie: {category}")
        print(f"🌐 Site: {site}")
        print(f"📊 Produits max: {max_products}")
        print(f"⭐ Reviews par note: {reviews_per_rating}")
        print()
        
        # Initialiser les stats
        self.session_stats['start_time'] = time.time()
        
        try:
            # Phase 1: Récupérer la liste des produits
            products = self._get_products_list(category, site, max_products)
            
            if not products:
                print("❌ Aucun produit trouvé")
                return pd.DataFrame()
            
            print(f"✅ {len(products)} produits trouvés")
            
            # Phase 2: Scraper les reviews de chaque produit
            all_reviews = []
            
            for i, product in enumerate(products, 1):
                print(f"\n📦 PRODUIT {i}/{len(products)}")
                print(f"🏷️ {product['title'][:60]}...")
                
                try:
                    product_reviews = self._scrape_product_reviews(
                        product, 
                        reviews_per_rating,
                        max_pages=5
                    )
                    
                    if product_reviews:
                        all_reviews.extend(product_reviews)
                        self.session_stats['reviews_collected'] += len(product_reviews)
                        print(f"✅ {len(product_reviews)} reviews récupérées")
                    else:
                        print("⚠️ Aucune review trouvée")
                    
                    self.session_stats['products_processed'] += 1
                    
                    # Délai humain entre produits
                    delay = random.uniform(3, 8)
                    print(f"⏳ Délai: {delay:.1f}s...")
                    time.sleep(delay)
                    
                except Exception as e:
                    print(f"❌ Erreur produit {i}: {e}")
                    self.session_stats['errors'] += 1
                    continue
            
            # Phase 3: Créer et nettoyer le DataFrame
            if all_reviews:
                df = pd.DataFrame(all_reviews)
                df = self._clean_review_data(df)
                
                # Stats finales
                duration = time.time() - self.session_stats['start_time']
                print("\n" + "="*80)
                print("📊 SCRAPING TERMINÉ - STATISTIQUES")
                print("="*80)
                print(f"⏱️ Durée: {duration/60:.1f} minutes")
                print(f"📦 Produits traités: {self.session_stats['products_processed']}")
                print(f"📝 Reviews récupérées: {self.session_stats['reviews_collected']}")
                print(f"❌ Erreurs: {self.session_stats['errors']}")
                print(f"📈 Taux de succès: {(1-self.session_stats['errors']/max(1,len(products)))*100:.1f}%")
                
                return df
            else:
                print("❌ Aucune review récupérée")
                return pd.DataFrame()
                
        except Exception as e:
            print(f"❌ Erreur scraping global: {e}")
            return pd.DataFrame()
    
    def _get_products_list(self, category, site, max_products):
        """Récupère la liste des produits à scraper"""
        
        try:
            # URL de recherche
            if site == 'amazon':
                search_url = f"https://www.amazon.com/s?k={category.replace(' ', '+')}"
            elif site == 'ebay':
                search_url = f"https://www.ebay.com/sch/i.html?_nkw={category.replace(' ', '+')}"
            else:
                print(f"❌ Site non supporté: {site}")
                return []
            
            print(f"🔍 Recherche: {search_url}")
            self.driver.get(search_url)
            
            # Attendre le chargement
            time.sleep(5)
            
            # Sélecteurs pour ce site
            site_selectors = self.selectors.get('products', {})
            
            if not site_selectors:
                print("❌ Sélecteurs produits non disponibles")
                return []
            
            # Récupérer les conteneurs de produits
            container_selector = site_selectors.get('container')
            if not container_selector:
                print("❌ Sélecteur conteneur manquant")
                return []
            
            containers = self.driver.find_elements(By.CSS_SELECTOR, container_selector)
            print(f"📦 {len(containers)} conteneurs trouvés")
            
            products = []
            
            for i, container in enumerate(containers[:max_products]):
                try:
                    product_data = self._extract_product_info(container, site_selectors, category)
                    
                    if product_data and product_data.get('url'):
                        products.append(product_data)
                        print(f"✅ Produit {len(products)}: {product_data['title'][:40]}...")
                    
                except Exception as e:
                    print(f"⚠️ Produit {i+1} ignoré: {e}")
                    continue
            
            return products
            
        except Exception as e:
            print(f"❌ Erreur récupération produits: {e}")
            return []
    
    def _extract_product_info(self, container, selectors, category):
        """Extrait les infos d'un produit depuis son conteneur"""
        
        product_data = {
            'category': category,
            'scraped_at': datetime.now().isoformat()
        }
        
        # Titre
        try:
            if selectors.get('title'):
                title_elem = container.find_element(By.CSS_SELECTOR, selectors['title'])
                product_data['title'] = title_elem.text.strip()
        except:
            product_data['title'] = 'Titre non trouvé'
        
        # URL
        try:
            if selectors.get('url'):
                url_elem = container.find_element(By.CSS_SELECTOR, selectors['url'])
                product_data['url'] = url_elem.get_attribute('href')
        except:
            product_data['url'] = None
        
        # Prix
        try:
            if selectors.get('price'):
                price_elem = container.find_element(By.CSS_SELECTOR, selectors['price'])
                product_data['price'] = price_elem.text.strip()
        except:
            product_data['price'] = 'N/A'
        
        # Rating
        try:
            if selectors.get('rating'):
                rating_elem = container.find_element(By.CSS_SELECTOR, selectors['rating'])
                rating_text = rating_elem.get_attribute('textContent') or rating_elem.text
                product_data['rating'] = rating_text.strip()
        except:
            product_data['rating'] = 'N/A'
        
        return product_data
    
    def _scrape_product_reviews(self, product, reviews_per_rating, max_pages=5):
        """Scrape les reviews d'un produit spécifique"""
        
        try:
            # Naviguer vers le produit
            self.driver.get(product['url'])
            time.sleep(3)
            
            # Trouver la page des reviews
            reviews_url = self._find_reviews_page(product['url'])
            
            if reviews_url:
                self.driver.get(reviews_url)
                time.sleep(3)
            else:
                print("⚠️ Page reviews non trouvée, tentative sur page produit")
            
            # Récupérer les reviews
            all_reviews = []
            
            # Sélecteurs reviews
            review_selectors = self.selectors.get('reviews', {})
            
            if not review_selectors:
                print("❌ Sélecteurs reviews non disponibles")
                return []
            
            # Scraper les reviews page par page
            for page in range(max_pages):
                try:
                    page_reviews = self._extract_reviews_from_page(product, review_selectors)
                    
                    if page_reviews:
                        all_reviews.extend(page_reviews)
                        print(f"📝 Page {page+1}: {len(page_reviews)} reviews")
                        
                        # Essayer de passer à la page suivante
                        if not self._go_to_next_page():
                            print("📄 Plus de pages disponibles")
                            break
                        
                        time.sleep(random.uniform(2, 4))
                    else:
                        print(f"📄 Page {page+1}: aucune review")
                        break
                        
                except Exception as e:
                    print(f"❌ Erreur page {page+1}: {e}")
                    break
            
            # Limiter le nombre de reviews si nécessaire
            if len(all_reviews) > reviews_per_rating * 5:  # 5 notes possibles
                all_reviews = all_reviews[:reviews_per_rating * 5]
            
            return all_reviews
            
        except Exception as e:
            print(f"❌ Erreur scraping reviews produit: {e}")
            return []
    
    def _find_reviews_page(self, product_url):
        """Trouve l'URL de la page des reviews"""
        
        try:
            # Sélecteurs de liens vers reviews
            review_link_selectors = [
                'a[href*="customer-reviews"]',
                'a[href*="reviews"]',
                '[data-hook="see-all-reviews-link-foot"]',
                '.cr-widget-ACR a'
            ]
            
            for selector in review_link_selectors:
                try:
                    links = self.driver.find_elements(By.CSS_SELECTOR, selector)
                    for link in links:
                        href = link.get_attribute('href')
                        if href and 'review' in href:
                            return href
                except:
                    continue
            
            # Si aucun lien trouvé, construire l'URL pour Amazon
            if 'amazon.com' in product_url:
                import re
                asin_match = re.search(r'/dp/([A-Z0-9]{10})', product_url)
                if asin_match:
                    asin = asin_match.group(1)
                    return f"https://www.amazon.com/product-reviews/{asin}"
            
            return None
            
        except Exception as e:
            print(f"⚠️ Erreur recherche page reviews: {e}")
            return None
    
    def _extract_reviews_from_page(self, product, review_selectors):
        """Extrait les reviews de la page actuelle"""
        
        reviews = []
        
        try:
            # Récupérer les conteneurs de reviews
            container_selector = review_selectors.get('container')
            if not container_selector:
                return []
            
            review_containers = self.driver.find_elements(By.CSS_SELECTOR, container_selector)
            
            for container in review_containers:
                try:
                    review_data = {
                        'product_name': product['title'],
                        'product_category': product['category'],
                        'product_url': product['url'],
                        'product_price': product.get('price', 'N/A'),
                        'scraped_at': datetime.now().isoformat()
                    }
                    
                    # Texte de la review
                    if review_selectors.get('text'):
                        try:
                            text_elem = container.find_element(By.CSS_SELECTOR, review_selectors['text'])
                            review_data['review_text'] = text_elem.text.strip()
                        except:
                            review_data['review_text'] = ''
                    
                    # Titre de la review
                    if review_selectors.get('title'):
                        try:
                            title_elem = container.find_element(By.CSS_SELECTOR, review_selectors['title'])
                            review_data['review_title'] = title_elem.text.strip()
                        except:
                            review_data['review_title'] = ''
                    
                    # Note de la review
                    if review_selectors.get('rating'):
                        try:
                            rating_elem = container.find_element(By.CSS_SELECTOR, review_selectors['rating'])
                            rating_text = rating_elem.get_attribute('textContent') or rating_elem.text
                            # Extraire le chiffre de la note
                            import re
                            rating_match = re.search(r'(\d+(?:\.\d+)?)', rating_text)
                            review_data['user_rating'] = float(rating_match.group(1)) if rating_match else None
                        except:
                            review_data['user_rating'] = None
                    
                    # Auteur
                    if review_selectors.get('author'):
                        try:
                            author_elem = container.find_element(By.CSS_SELECTOR, review_selectors['author'])
                            review_data['reviewer_name'] = author_elem.text.strip()
                        except:
                            review_data['reviewer_name'] = 'Anonymous'
                    
                    # Date
                    if review_selectors.get('date'):
                        try:
                            date_elem = container.find_element(By.CSS_SELECTOR, review_selectors['date'])
                            review_data['review_date'] = date_elem.text.strip()
                        except:
                            review_data['review_date'] = 'N/A'
                    
                    # Ajouter si on a du contenu utile
                    if (review_data.get('review_text') and len(review_data['review_text']) > 10) or \
                       (review_data.get('review_title') and len(review_data['review_title']) > 5):
                        reviews.append(review_data)
                    
                except Exception as e:
                    continue  # Ignorer les reviews problématiques
            
            return reviews
            
        except Exception as e:
            print(f"❌ Erreur extraction reviews: {e}")
            return []
    
    def _go_to_next_page(self):
        """Tente de passer à la page suivante"""
        
        try:
            # Sélecteurs de bouton "suivant"
            next_selectors = [
                '.a-pagination .a-last a',
                'a[aria-label="Next page"]',
                '.a-pagination li:last-child a',
                'a[href*="pageNumber"]'
            ]
            
            for selector in next_selectors:
                try:
                    next_button = self.driver.find_element(By.CSS_SELECTOR, selector)
                    if next_button.is_enabled():
                        next_button.click()
                        time.sleep(2)
                        return True
                except:
                    continue
            
            return False
            
        except Exception as e:
            print(f"⚠️ Erreur navigation page suivante: {e}")
            return False
    
    def _clean_review_data(self, df):
        """Nettoie et optimise les données récupérées"""
        
        try:
            print("🧹 Nettoyage des données...")
            
            # Supprimer les doublons
            initial_count = len(df)
            df = df.drop_duplicates(subset=['review_text', 'product_name'], keep='first')
            print(f"📝 Doublons supprimés: {initial_count - len(df)}")
            
            # Nettoyer les textes
            df['review_text'] = df['review_text'].str.strip()
            df['review_title'] = df['review_title'].str.strip()
            df['product_name'] = df['product_name'].str.strip()
            
            # Filtrer les reviews trop courtes
            df = df[df['review_text'].str.len() > 15]
            
            # Ajouter des métriques
            df['review_length'] = df['review_text'].str.len()
            df['word_count'] = df['review_text'].str.split().str.len()
            
            # Nettoyer les ratings
            df['user_rating'] = pd.to_numeric(df['user_rating'], errors='coerce')
            
            print(f"✅ {len(df)} reviews nettoyées")
            return df
            
        except Exception as e:
            print(f"❌ Erreur nettoyage: {e}")
            return df
    
    def save_data(self, df, filename=None):
        """Sauvegarde les données avec horodatage"""
        
        try:
            if filename is None:
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"../data/raw/reviews_robustes_{timestamp}.csv"
            
            import os
            os.makedirs(os.path.dirname(filename), exist_ok=True)
            
            df.to_csv(filename, index=False, encoding='utf-8')
            print(f"💾 Données sauvegardées: {filename}")
            
            # Sauvegarder aussi en JSON pour backup
            json_filename = filename.replace('.csv', '.json')
            df.to_json(json_filename, orient='records', force_ascii=False, indent=2)
            
            return filename
            
        except Exception as e:
            print(f"❌ Erreur sauvegarde: {e}")
            return None
    
    def close(self):
        """Ferme proprement le scraper"""
        if self.driver:
            try:
                self.driver.quit()
                print("✅ Driver scraper fermé")
            except:
                pass
            self.driver = None

print("✅ Scraper robuste créé!")

✅ Scraper robuste créé!


In [4]:
# ============================================================================
# WORKFLOW PRINCIPAL ROBUSTE
# ============================================================================

def robust_reviews_workflow(category="laptop", site="amazon", max_products=10, reviews_per_rating=50, headless=False):
    """
    Workflow principal ultra-robuste pour scraper les reviews de produits
    
    Phase 1: Détection automatique des balises (Scout)
    Phase 2: Scraping des reviews avec balises validées (Scraper)
    
    Args:
        category: catégorie de produits (ex: "laptop", "smartphone")
        site: site à scraper ("amazon" ou "ebay")
        max_products: nombre max de produits à analyser
        reviews_per_rating: nombre de reviews à récupérer par note
        headless: mode sans interface (True) ou visible (False)
    """
    
    print("="*100)
    print("🚀 WORKFLOW ROBUSTE - SCRAPING REVIEWS DE PRODUITS")
    print("="*100)
    print(f"📦 Catégorie: {category}")
    print(f"🌐 Site: {site}")
    print(f"📊 Produits max: {max_products}")
    print(f"⭐ Reviews par note: {reviews_per_rating}")
    print(f"👁️ Mode: {'Headless' if headless else 'Visible'}")
    print(f"📈 Total estimé: {max_products * 5 * reviews_per_rating} reviews max")
    print()
    
    # Variables pour cleanup
    scout = None
    scraper = None
    
    try:
        # ====================================================================
        # PHASE 1: DÉTECTION DES BALISES (SCOUT)
        # ====================================================================
        print("🔍 PHASE 1: DÉTECTION AUTOMATIQUE DES BALISES")
        print("-" * 70)
        
        scout = RobustProductReviewScout()
        
        # Setup du driver scout
        if not scout.setup_robust_driver(headless=True, timeout=30):
            print("❌ Impossible d'initialiser le scout")
            return None
        
        # Détection des sélecteurs
        site_url = f"https://www.{site}.com"
        detected_selectors = scout.detect_site_selectors(site_url, category)
        
        if not detected_selectors or not detected_selectors.get('products'):
            print("❌ Échec de la détection des sélecteurs")
            scout.close()
            return None
        
        print("✅ Sélecteurs détectés avec succès!")
        print(f"📦 Produits: {list(detected_selectors['products'].keys())}")
        print(f"📝 Reviews: {list(detected_selectors['reviews'].keys())}")
        
        # Fermer le scout
        scout.close()
        scout = None
        
        print("\n" + "="*70)
        
        # ====================================================================
        # PHASE 2: SCRAPING DES REVIEWS (SCRAPER)
        # ====================================================================
        print("📊 PHASE 2: SCRAPING DES REVIEWS")
        print("-" * 70)
        
        scraper = RobustProductReviewScraper()
        scraper.selectors = detected_selectors  # Utiliser les sélecteurs détectés
        
        # Setup du driver scraper
        if not scraper.setup_robust_driver(headless=headless, timeout=60):
            print("❌ Impossible d'initialiser le scraper")
            return None
        
        # Avertissement utilisateur
        if not headless:
            print("\n🚨 AVERTISSEMENT: Scraping sur site réel en cours!")
            print("⏰ Durée estimée: {:.1f} minutes".format(max_products * 3))
            print("📝 Respectez les ToS et les limitations de débit")
            
            response = input("\n🔄 Continuer? (o/n): ").strip().lower()
            if response not in ['o', 'oui', 'y', 'yes', '']:
                print("⏹️ Arrêt demandé par l'utilisateur")
                scraper.close()
                return None
        
        # Lancement du scraping
        print(f"\n🎯 Début du scraping pour '{category}' sur {site}...")
        
        df_reviews = scraper.scrape_category_reviews(
            category=category,
            site=site,
            max_products=max_products,
            reviews_per_rating=reviews_per_rating
        )
        
        # ====================================================================
        # PHASE 3: RÉSULTATS ET SAUVEGARDE
        # ====================================================================
        if df_reviews.empty:
            print("❌ Aucune review récupérée")
            scraper.close()
            return None
        
        print("\n" + "="*70)
        print("📊 ANALYSE DES RÉSULTATS")
        print("-" * 70)
        
        # Statistiques détaillées
        stats = {
            'total_reviews': len(df_reviews),
            'unique_products': df_reviews['product_name'].nunique(),
            'avg_review_length': df_reviews['review_length'].mean(),
            'rating_distribution': df_reviews['user_rating'].value_counts().sort_index(),
            'top_products': df_reviews['product_name'].value_counts().head(5)
        }
        
        print(f"✅ Total reviews: {stats['total_reviews']}")
        print(f"📦 Produits uniques: {stats['unique_products']}")
        print(f"📝 Longueur moyenne: {stats['avg_review_length']:.0f} caractères")
        print(f"⭐ Distribution des notes:")
        for rating, count in stats['rating_distribution'].items():
            if pd.notna(rating):
                print(f"   {rating} étoiles: {count} reviews")
        
        print(f"\n🏆 Top produits par nombre de reviews:")
        for product, count in stats['top_products'].items():
            print(f"   • {product[:50]}... ({count} reviews)")
        
        # Sauvegarde
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"../data/raw/{site}_{category}_{timestamp}.csv"
        
        saved_file = scraper.save_data(df_reviews, filename)
        
        # Aperçu des données
        print(f"\n📋 APERÇU DES DONNÉES (5 premières reviews):")
        print("-" * 70)
        
        sample_cols = ['product_name', 'user_rating', 'review_text', 'reviewer_name']
        available_cols = [col for col in sample_cols if col in df_reviews.columns]
        
        for i, row in df_reviews[available_cols].head(5).iterrows():
            print(f"\nReview {i+1}:")
            for col in available_cols:
                value = str(row[col])
                if col == 'review_text' and len(value) > 100:
                    value = value[:100] + "..."
                elif col == 'product_name' and len(value) > 50:
                    value = value[:50] + "..."
                print(f"  {col}: {value}")
        
        # Fermer le scraper
        scraper.close()
        scraper = None
        
        print("\n" + "="*100)
        print("🎉 WORKFLOW TERMINÉ AVEC SUCCÈS!")
        print("="*100)
        
        return df_reviews
        
    except KeyboardInterrupt:
        print("\n⏹️ Arrêt demandé par l'utilisateur")
        return None
        
    except Exception as e:
        print(f"\n❌ Erreur workflow: {e}")
        import traceback
        traceback.print_exc()
        return None
        
    finally:
        # Cleanup des ressources
        if scout:
            try:
                scout.close()
            except:
                pass
        if scraper:
            try:
                scraper.close()
            except:
                pass

def quick_scout_test(site="amazon", category="laptop"):
    """
    Test rapide du scout uniquement
    """
    print(f"🧪 TEST RAPIDE - Scout pour {category} sur {site}")
    print("-" * 50)
    
    scout = RobustProductReviewScout()
    
    try:
        if scout.setup_robust_driver(headless=True):
            site_url = f"https://www.{site}.com"
            selectors = scout.detect_site_selectors(site_url, category)
            
            if selectors:
                print("✅ Test scout réussi!")
                print(f"📦 Sélecteurs produits: {list(selectors['products'].keys())}")
                print(f"📝 Sélecteurs reviews: {list(selectors['reviews'].keys())}")
                return selectors
            else:
                print("❌ Test scout échoué")
                return None
        else:
            print("❌ Impossible de créer le driver scout")
            return None
            
    except Exception as e:
        print(f"❌ Erreur test scout: {e}")
        return None
        
    finally:
        scout.close()

def robust_reviews_menu():
    """
    Menu principal pour le workflow robuste
    """
    print("="*100)
    print("🎯 WORKFLOW ROBUSTE - REVIEWS DE PRODUITS")
    print("="*100)
    print()
    print("1️⃣ Workflow complet (scout + scraper)")
    print("2️⃣ Test scout uniquement")
    print("3️⃣ Configuration avancée")
    print("4️⃣ Voir fichiers de données")
    print("5️⃣ Quitter")
    print()
    
    while True:
        try:
            choice = input("👉 Votre choix (1-5): ").strip()
            
            if choice == '1':
                print("\n📋 CONFIGURATION DU WORKFLOW COMPLET")
                print("-" * 50)
                
                category = input("🏷️ Catégorie (ex: laptop, smartphone): ").strip() or "laptop"
                site = input("🌐 Site (amazon/ebay): ").strip() or "amazon"
                
                try:
                    max_products = int(input("📦 Nombre de produits (défaut: 5): ") or "5")
                    reviews_per_rating = int(input("⭐ Reviews par note (défaut: 20): ") or "20")
                except ValueError:
                    max_products, reviews_per_rating = 5, 20
                
                headless_choice = input("👁️ Mode headless? (o/n, défaut: n): ").strip().lower()
                headless = headless_choice in ['o', 'oui', 'y', 'yes']
                
                print(f"\n🚀 Lancement du workflow...")
                result = robust_reviews_workflow(category, site, max_products, reviews_per_rating, headless)
                
                if result is not None:
                    print(f"\n✅ Workflow terminé - {len(result)} reviews récupérées")
                else:
                    print("\n❌ Workflow échoué")
                
                return result
                
            elif choice == '2':
                print("\n🧪 TEST SCOUT")
                print("-" * 30)
                
                site = input("🌐 Site (amazon/ebay): ").strip() or "amazon"
                category = input("🏷️ Catégorie: ").strip() or "laptop"
                
                result = quick_scout_test(site, category)
                if result:
                    print("✅ Scout fonctionne correctement!")
                else:
                    print("❌ Problème avec le scout")
                
            elif choice == '3':
                print("\n⚙️ CONFIGURATION AVANCÉE")
                print("-" * 40)
                print("📖 Paramètres disponibles dans robust_reviews_workflow():")
                print("   • category: catégorie de produits")
                print("   • site: amazon ou ebay")
                print("   • max_products: nombre de produits max")
                print("   • reviews_per_rating: reviews par note (1-5)")
                print("   • headless: mode sans interface")
                print("\n💡 Exemple:")
                print("   df = robust_reviews_workflow('gaming laptop', 'amazon', 8, 30, False)")
                
            elif choice == '4':
                print("\n📁 FICHIERS DE DONNÉES")
                print("-" * 30)
                
                import os
                data_dir = "../data/raw"
                
                if os.path.exists(data_dir):
                    files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
                    if files:
                        print("📄 Fichiers CSV trouvés:")
                        for f in sorted(files, reverse=True)[:10]:  # 10 plus récents
                            size = os.path.getsize(os.path.join(data_dir, f)) / 1024  # KB
                            print(f"   • {f} ({size:.1f} KB)")
                    else:
                        print("❌ Aucun fichier CSV trouvé")
                else:
                    print("❌ Dossier data/raw non trouvé")
                
            elif choice == '5':
                print("👋 À bientôt!")
                break
                
            else:
                print("❌ Choix invalide")
                
        except KeyboardInterrupt:
            print("\n👋 À bientôt!")
            break
        except Exception as e:
            print(f"❌ Erreur: {e}")
    
    return None

# Configuration d'exemples prêts à l'emploi
ROBUST_EXAMPLES = {
    'laptops_gaming': {
        'category': 'gaming laptop',
        'site': 'amazon',
        'max_products': 8,
        'reviews_per_rating': 25,
        'headless': False
    },
    'smartphones': {
        'category': 'smartphone',
        'site': 'amazon', 
        'max_products': 6,
        'reviews_per_rating': 30,
        'headless': False
    },
    'headphones': {
        'category': 'wireless headphones',
        'site': 'amazon',
        'max_products': 10,
        'reviews_per_rating': 20,
        'headless': False
    }
}

def run_example(example_name):
    """Exécute un exemple prédéfini"""
    if example_name in ROBUST_EXAMPLES:
        config = ROBUST_EXAMPLES[example_name]
        print(f"🚀 Lancement exemple: {example_name}")
        return robust_reviews_workflow(**config)
    else:
        print(f"❌ Exemple '{example_name}' non trouvé")
        print(f"📋 Disponibles: {list(ROBUST_EXAMPLES.keys())}")
        return None

print("✅ Workflow robuste prêt!")
print("📖 Utilisez robust_reviews_menu() pour commencer")
print("🚀 Ou run_example('laptops_gaming') pour un test rapide")

✅ Workflow robuste prêt!
📖 Utilisez robust_reviews_menu() pour commencer
🚀 Ou run_example('laptops_gaming') pour un test rapide


In [7]:
robust_reviews_menu()

🎯 WORKFLOW ROBUSTE - REVIEWS DE PRODUITS

1️⃣ Workflow complet (scout + scraper)
2️⃣ Test scout uniquement
3️⃣ Configuration avancée
4️⃣ Voir fichiers de données
5️⃣ Quitter


📋 CONFIGURATION DU WORKFLOW COMPLET
--------------------------------------------------

🚀 Lancement du workflow...
🚀 WORKFLOW ROBUSTE - SCRAPING REVIEWS DE PRODUITS
📦 Catégorie: laptop
🌐 Site: amazon
📊 Produits max: 5
⭐ Reviews par note: 20
👁️ Mode: Headless
📈 Total estimé: 500 reviews max

🔍 PHASE 1: DÉTECTION AUTOMATIQUE DES BALISES
----------------------------------------------------------------------
🔧 Tentative 1/3 - Setup driver scout...


2025-06-29 16:24:30,659 - INFO - patching driver executable C:\Users\Yann\appdata\roaming\undetected_chromedriver\undetected_chromedriver.exe


✅ Driver scout initialisé avec succès!
🔍 Détection des sélecteurs pour amazon...
🌐 Navigation vers: https://www.amazon.com/s?k=laptop
❌ Aucun conteneur de produit trouvé
❌ Impossible de détecter les sélecteurs produits
❌ Échec de la détection des sélecteurs
✅ Driver scout fermé

❌ Workflow échoué
